# Optimizing Strategy Parameters

In [ ]:
???

<table style="width:100%; height:90%">
      <tr>
    <th>Parametrize the Strategy</th>
    <th>Optimizing Limits' Parameters</th>
  </tr>
  <tr>
    <td><img src="src/07_Code_Regression Strategy Limits X.png" alt="Parametrize the Strategy" style="width:100%"></td>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [1]:
import pickle

with open('models/my_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [3]:
import pandas as pd
df = pd.read_excel('data/microsoft-linkedin-processed.xlsx' , index_col=0)
df.index = pd.to_datetime(df.index)


# Simple Investment Strategy

### Create Strategy class

In [4]:
from backtesting import Strategy, Backtest

In [7]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [8]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [9]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting with specific values

In [ ]:
results = bt.run()


Start                     2016-12-01 00:00:00
End                       2024-11-08 00:00:00
Duration                   2899 days 00:00:00
Exposure Time [%]                     99.8999
Equity Final [$]                 76523.057577
Equity Peak [$]                  88428.310111
Return [%]                         665.230576
Buy & Hold Return [%]              613.750005
Return (Ann.) [%]                   29.261533
Volatility (Ann.) [%]               34.047169
Sharpe Ratio                         0.859441
Sortino Ratio                        1.716706
Calmar Ratio                         0.894669
Max. Drawdown [%]                  -32.706557
Avg. Drawdown [%]                   -2.916093
Max. Drawdown Duration      496 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   22
Win Rate [%]                        54.545455
Best Trade [%]                      90.545124
Worst Trade [%]                    -10.559322
Avg. Trade [%]                    

### Interpret backtesting results

In [11]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-01 00:00:00
End,2024-11-08 00:00:00
Duration,2899 days 00:00:00
Exposure Time [%],99.8999
Equity Final [$],76523.057577
Equity Peak [$],88428.310111
Return [%],665.230576


## Parametrize the Investment Strategy

### Create Strategy class

In [12]:
from backtesting import Strategy, Backtest

In [13]:
class Regression(Strategy):
    limit_buy = 3
    limit_sell = -2
    
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [15]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)
results = bt.run()
results

Start                     2016-12-01 00:00:00
End                       2024-11-08 00:00:00
Duration                   2899 days 00:00:00
Exposure Time [%]                   85.185185
Equity Final [$]                  9722.872487
Equity Peak [$]                  20945.926054
Return [%]                          -2.771275
Buy & Hold Return [%]              613.750005
Return (Ann.) [%]                   -0.353837
Volatility (Ann.) [%]               29.523952
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -62.704942
Avg. Drawdown [%]                   -4.389555
Max. Drawdown Duration     1733 days 00:00:00
Avg. Drawdown Duration       65 days 00:00:00
# Trades                                   40
Win Rate [%]                             45.0
Best Trade [%]                      71.320181
Worst Trade [%]                    -20.534079
Avg. Trade [%]                    

## Optimize backtesting with multiple combinations

In [16]:
list_limits_buy = list(range(0, 11, 1))
list_limits_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [17]:
list_limits_sell = list(range(0, -11, -1))
list_limits_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [27]:
%%time

results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Equity Peak [$]', return_heatmap=True
)

CPU times: user 4.03 s, sys: 27.8 ms, total: 4.06 s
Wall time: 3min 32s


In [19]:
results[0]

Start                     2016-12-01 00:00:00
End                       2024-11-08 00:00:00
Duration                   2899 days 00:00:00
Exposure Time [%]                     99.8999
Equity Final [$]               2209827.825089
Equity Peak [$]                 2210399.19899
Return [%]                       21998.278251
Buy & Hold Return [%]              613.750005
Return (Ann.) [%]                   97.553554
Volatility (Ann.) [%]               52.496232
Sharpe Ratio                         1.858296
Sortino Ratio                        6.297318
Calmar Ratio                         4.727277
Max. Drawdown [%]                  -20.636309
Avg. Drawdown [%]                   -2.461832
Max. Drawdown Duration      111 days 00:00:00
Avg. Drawdown Duration       12 days 00:00:00
# Trades                                  387
Win Rate [%]                        65.891473
Best Trade [%]                      17.813153
Worst Trade [%]                     -12.89161
Avg. Trade [%]                    

In [20]:
results[1]

limit_buy  limit_sell
0           0            21998.278251
           -1            15408.117204
           -2             7551.948918
           -3             3626.654669
           -4             1992.964989
                             ...     
10         -6             -100.000000
           -7             -100.000000
           -8             -100.000000
           -9             -100.000000
           -10            -100.000000
Name: Return [%], Length: 121, dtype: float64

### [ ] Interpret optimization results

In [21]:
dff = results[1].reset_index()
dff

,limit_buy,limit_sell,Return [%]
0,0,0,21998.278251
1,0,-1,15408.117204
2,0,-2,7551.948918
3,0,-3,3626.654669
4,0,-4,1992.964989
...,...,...,...
116,10,-6,-100.000000
117,10,-7,-100.000000
118,10,-8,-100.000000
119,10,-9,-100.000000


In [25]:
dff = dff.pivot(index="limit_buy", columns="limit_sell", values="Return [%]")

KeyError: 'limit_buy'

### DataFrame heatmaps for better reporting

In [26]:
dff.sort_index(axis=1, ascending=False)\
  .style.format(precision=0)\
    .background_gradient(vmin=dff.values.min(), vmax=dff.values.max())

limit_buy,10,9,8,7,6,5,4,3,2,1,0
limit_sell,,,,,,,,,,,
-10,-100,-100,169,186,348,348,348,368,578,582,582
-9,-100,-100,169,186,348,348,348,368,578,582,582
-8,-100,-100,-100,-100,149,149,251,266,659,663,608
-7,-100,-100,-100,-100,119,119,211,223,684,688,632
-6,-100,-100,-100,-100,58,58,100,108,747,758,742
-5,-100,-100,-100,-100,88,96,145,105,735,665,967
-4,-100,-100,-100,-100,54,64,147,109,1766,1240,1993
-3,-100,-100,-100,-100,-100,-31,-1,22,698,969,3627
-2,-100,-100,-100,-100,-100,-25,-18,-3,656,1537,7552
